In [93]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd

In [170]:
df = pd.DataFrame({
    'Year': [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974, 1978, 1982,
             1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014, 2018, 2022],
    'Winners': ['Uruguay', 'Italy', 'Italy', 'Uruguay', 'Germany', 'Brazil', 'Brazil', 'England', 
                'Brazil', 'Germany', 'Argentina', 'Italy', 'Argentina', 'Germany', 'Brazil', 
                'France', 'Brazil', 'Italy', 'Spain', 'Germany', 'France', 'Argentina'],
    'Runner_ups': ['Argentina', 'Czechoslovakia', 'Hungary', 'Brazil', 'Hungary', 'Sweden', 
                   'Czechoslovakia', 'Germany', 'Italy', 'Netherlands', 'Netherlands',
                   'Germany', 'Germany', 'Argentina', 'Italy', 'Brazil', 'Germany',
                   'France', 'Netherlands', 'Argentina', 'Croatia', 'France']
})

win_counts = df['Winners'].value_counts().reset_index()
win_counts.columns = ['Country', 'Wins']

fig1 = px.choropleth(
    win_counts,
    locations='Country',
    locationmode='country names',
    color='Wins',
    color_continuous_scale="peach",
    scope='world',
    title="World Cup Results"
)

country_options = [{'label': country, 'value': country} for country in df['Winners'].unique()]
year_options = [{'label': str(y), 'value': y} for y in sorted(df['Year'].unique())]


app = dash.Dash(__name__)
app.title = "World Cup"

app.layout = html.Div([
    html.H1("FIFA World Cup Results"),
    dcc.Graph(figure=fig1),

    html.Div([
        html.Label("Select a Country:"),
        dcc.Dropdown(
            id='country-dropdown',
            options=country_options,
            value='Brazil'
        ),
        html.Div(id='win-output', style={'marginTop':'20px', 'fontSize':'20px'})
    ], style={'width':'40%', 'margin':'auto', 'marginTop':'30px'}),

    html.Div([
        html.Label("Select a Year:"),
        dcc.Dropdown(
            id='year-dropdown',
            options=year_options,
            value=2022
        ),
        html.Div(id='match-output', style={'marginTop':'20px', 'fontSize':'20px'})
    ], style={'width':'40%', 'margin':'auto', 'marginTop':'30px'}),
])

@app.callback(
    Output('match-output', 'children'),
    Input('year-dropdown', 'value'),
)

def update_match_result(selected_year):
    row = df[df['Year'] == selected_year]
    if not row.empty:
        winner = row.iloc[0]['Winners']
        runner_up = row.iloc[0]['Runner_ups']
        return f"In the {selected_year} FIFA World Cup, the winner was {winner} and the runner-up was {runner_up}."
    else:
        return "No data for that year."

app.run(debug=True, port=8053)